In [ ]:
# Strategy:
# - Compute feature-space centroid of each training class
# - Take directional vector between centroids as "x" axis
# - Take weight vector as "z" axis
# - Compute cross product as "y" axis
def feature_space_axes(model, images_a, images_b):
    feature_model = get_penultimate_layer_model(model)
    
    feat_a = feature_model(images_a,training=False)
    feat_b = feature_model(images_b,training=False)
    
    mu_a = feat_a.numpy().mean(axis=0)
    mu_b = feat_b.numpy().mean(axis=0)
    delta_mu = mu_a - mu_b
    
    axis_x = delta_mu / np.linalg.norm(delta_mu)
    
    output_weights = (model.layers[-1].weights[0]).numpy().squeeze()
    
    support = output_weights - axis_x * np.dot(output_weights,axis_x)
    
    axis_y = support / np.linalg.norm(support)
    
    return (axis_x, axis_y)
    
    
def group_by_classes(dataset):
    ii = []
    ll = []
    
    for (img,label) in dataset:
        ii.append(img)
        ll.append(label)
    
    
    
    all_img = np.concatenate(ii)
    all_label = np.concatenate(ll)
    
    g = {}
    
    for k in np.unique(all_label):
        g[k] = all_img[all_label==k]
    return g

def plot_2d_proj(modegl,  dir_x, dir_y, dataset, img_t):
    basis = np.column_stack((dir_x,dir_y))
    
    feature_model = get_penultimate_layer_model(modegl)
    
    xy = []
    ll = []
    
    for(img,label) in dataset:
        feat = feature_model(img,training=False)
        xy.append(np.matmul(feat.numpy(),basis))
        ll.append(label)
    
    points = np.concatenate(xy)
    labels = np.concatenate(ll)
    
    
    plt.scatter(points[:,0],points[:,1],c=labels)
    
    feat_t = feature_model(img_t,training=False)
    xy_t = np.matmul(feat_t.numpy(),basis)
    
    plt.scatter(xy_t[:,0], xy_t[:,1], color="pink")
    
    
        

In [ ]:
_grouped = group_by_classes(train_dataset_p)
(_axis_x, _axis_y) = feature_space_axes(model_poisoned,_grouped[0][:100],_grouped[1][:100])
plot_2d_proj(model,_axis_x,_axis_y, train_dataset_p, _poison_image)
plot_2d_proj(model_poisoned,_axis_x,_axis_y, train_dataset_p, _poison_image)